# Dynamics of Species Reintroduciton

In [1]:
import os
while not (os.getcwd().split('\\')[-1] == 'ecological-networks' or os.getcwd().split('/')[-1] == 'ecological-networks'):
    %cd ..
import setup_paths
setup_paths.add_path()
from Species_Reintroductions import *
global DATA_TYPE
DATA_TYPE = 'Real_1D'
CASE_TO_SOLVE_init(DATA_TYPE)
get_CASE()

def CASE_init(CASE_TO_SOLVE):
    DATA_TYPE = CASE_TO_SOLVE
    CASE_TO_SOLVE_init(DATA_TYPE)
    NetworkName_Solved_df = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"Network_Solved_{DATA_TYPE.split('_')[0]}.pkl"))
    M = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"M_{DATA_TYPE.split('_')[0]}.pkl"))
    if DATA_TYPE.split('_')[0] == 'Syn':
        M = {k:M[k] for k in M.keys() if int(k.split('_')[2]) == 100}
    return DATA_TYPE, NetworkName_Solved_df, M

from scipy.optimize import fsolve
from scipy import stats
import scipy.linalg as la
from sklearn.linear_model import LinearRegression


/home/udit/Desktop/Sarth/ecological-networks/Code/01_Species_Reintroductions
/home/udit/Desktop/Sarth/ecological-networks/Code
/home/udit/Desktop/Sarth/ecological-networks
Real_1D


In [ ]:
def ODE_Solve_func(A,t0=0,tf=100,num=30,x_low=10e-6, P = {i:j for i,j in zip(['a', 'b', 'mu', 'h', 'g0', 't', 'k'],[0.3, 1.0, 1e-4, 0.7, 1.0, 0.5, 0.0])}, C_type = 'UA'):
    t = np.linspace(t0,tf,num)
    n,m = A.shape
    x0 = [x_low, x_low]
    g_p, g_a = [0.0,0.0]
    Degree_P = list(np.sum(np.array(A), axis = 1))
    Degree_A = list(np.sum(np.array(A), axis = 0))
    if C_type == 'UA':
        # Unweighted Averaging
        g_p = np.sum(P['g0'] * np.power(np.array(Degree_P),1-P['t'])) / len(Degree_P)
        g_a = np.sum(P['g0'] * np.power(np.array(Degree_A),1-P['t'])) / len(Degree_A)
    elif C_type == 'DA':
        # Degree Weihted Averaging
        g_p = np.sum(P['g0'] * np.power(np.array(Degree_P),2-P['t'])) / sum(Degree_P)
        g_a = np.sum(P['g0'] * np.power(np.array(Degree_A),2-P['t'])) / sum(Degree_A)
    elif C_type == 'EA':
    # Eigenvector based Averaging
        M_P = np.matmul(np.array(M_df),np.array(M_df.T))
        M_A = np.matmul(np.array(M_df.T),np.array(M_df))
        _, V_P = la.eig(M_P)
        _, V_A = la.eig(M_A)
        g_p = np.sum( P['g0'] * np.multiply( np.power(np.array(Degree_P),1-P['t']), list(V_P[:,0].real) ) ) / sum(list(V_P[:,0].real))
        g_a = np.sum( P['g0'] * np.multiply( np.power(np.array(Degree_A),1-P['t']), list(V_A[:,0].real) ) ) / sum(list(V_A[:,0].real))

    def eqs(x, t, x0, P, g_p, g_a):
        Pe, Ae = x
        return [ P['a']*Pe - P['b']*(Pe**2) + P['mu'] + (g_p*Ae*Pe)/(1 + P['h']*g_p*Ae), 
                (P['a']-P['k'])*Ae - P['b']*(Ae**2) + P['mu'] + (g_a*Pe*Ae)/(1 + P['h']*g_a*Pe) ]
    xl = odeint(eqs,x0,t,args=(A, P, g_p, g_a, ))
    return xl
P = {i:j for i,j in zip(['a', 'b', 'mu', 'h', 'g0', 't', 'k'],[0.3, 1.0, 1e-4, 0.7, 1.0, 0.5, 0.0])}
DATA_TYPE, _, M = CASE_init('Real_2D')
NetworkName = 'M_PL_041'
M_df = M[NetworkName]
X = ODE_Solve_func(
    A = M_df,
    t0=0,
    tf=100,
    num=100,
    x_low=10e-6, 
    P = P,
    C_type = 'UA'
    )
print(X[-1,:])
plt.plot(X[:,0],'--b')
plt.plot(X[:,1],'--r')

In [ ]:
from mpl_toolkits import mplot3d
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

def f(g_p, g_a):
    t = np.linspace(0,100,100)
    x0 = [10e-6, 10e-6]
    P = {i:j for i,j in zip(['a', 'b', 'mu', 'h', 'g0', 't', 'k'],[0.3, 1.0, 1e-4, 0.7, 1.0, 0.5, 0.0])}
    def eqs(x, t):
        Pe, Ae = x
        return [ P['a']*Pe - P['b']*(Pe**2) + P['mu'] + (g_p*Ae*Pe)/(1 + P['h']*g_p*Ae), 
                (P['a']-P['k'])*Ae - P['b']*(Ae**2) + P['mu'] + (g_a*Pe*Ae)/(1 + P['h']*g_a*Pe) ]
    XL = odeint(eqs,x0,t)
    Peff, Aeff = XL[-1,:]
    return Peff+Aeff

x = np.arange(0.0, 10.0, 0.5) # g_p
y = np.arange(0.0, 10.0, 0.5) # g_a

X, Y = np.meshgrid(x, y)
xf = X.flatten()
yf = Y.flatten()
Z = [f(xx,yy) for xx,yy in zip(xf,yf)]

Z = np.reshape(Z, X.shape)
#'''
fig = plt.figure(figsize = (10,10))
'''
ax = plt.axes(projection='3d')
#ax.contour3D(X, Y, Z, 50)
ax.plot_surface(X,Y,Z)
ax.set_xlabel('g_p')
ax.set_ylabel('g_a')
#ax.set_zlabel('<X>')
'''
#'''
ax = fig.add_axes([1,1,1,1])
cp = ax.contourf(X,Y,Z)
fig.colorbar(cp)
#ax.plot_surface(X,Y,Z)
ax.set_xlabel(r'$<\gamma_P>$', fontsize = 20)
ax.set_ylabel(r'$<\gamma_A>$', fontsize = 20)
#'''


In [ ]:
from mpl_toolkits import mplot3d
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
DATA_TYPE, _, M = CASE_init('Real_1D')
def f(beta_eff, x0 = 1.0):
    t = np.linspace(0,100,100)
    P = {i:j for i,j in zip(['B', 'C', 'D', 'E', 'H', 'K'],[0.1, 1.0, 5.0, 0.9, 0.1, 5])}
    def eqs(x, t):
        return P['B'] + x*(1 - (x/P['K']))*((x/P['C'])-1) + beta_eff*((x**2)/(P['D'] + (P['E']+P['H'])*x)) 
    X_eff = odeint(eqs,x0,t)
    return X_eff
'''
for beta_eff in np.arange(0.2, 2.0, 0.1):
    plt.figure(figsize = (10,10))
    for x0 in np.arange(0.0, 4.0, 0.1):
        X = f(beta_eff, x0)
        plt.plot(X)
'''
x = np.arange(0.0, 10.0, 0.1) # beta_eff
y = np.array([f(xx)[-1,0] for xx in x]) # <X>
plt.figure()
plt.plot(x,y)
plt.xlabel(r'$\beta_{eff}$', fontsize = 20)
plt.ylabel(r'$<X>$', fontsize = 20)


In [9]:
def G_iter(data_two):
    for key in list(data_two.keys()):
        for ensembles_ID in range(len(data_two[key])):
            yield (key, ensembles_ID, 0)

def get_Data(NetworkName, loc):
    Data_Scatter = []
    #print(NetworkName)
    M_df = M[NetworkName]
    #data_two = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '02_Generate_Database', f"{DATA_TYPE}_Data", f"{NetworkName}", f"{NetworkName}-Data_two.pkl"))
    #data_two = UnpickleObj(os.path.join(r"D:\WORK\EN\Documents\Ecological_Restoration_01\Real_Data", f"{NetworkName}\{NetworkName}-Data_two.pkl"))
    data_two = UnpickleObj(f"/home/udit/Documents/Ecological_Restoration_{loc}/{DATA_TYPE.split('_')[0]}_Data/{NetworkName}/{NetworkName}-Data_two.pkl")
    Keys_Filter = [[0,1,2], [2,5,8], [0,2]]
    Ensembles_batch = [0,10]
    data_two = {k: v[Ensembles_batch[0]:min(Ensembles_batch[1],len(v))] for k,v in data_two.items() if k in set(list(itertools.product(*Keys_Filter)))}
    del Keys_Filter, Ensembles_batch
    P = {i:j for i,j in zip(['a', 'b', 'mu', 'h', 'g0', 't', 'k'],[0.3, 1.0, 1e-4, 0.7, 1.0, 0.5, 0.0])}
    for approach in ['three','four','five','six','rand']:
        for key, E, T in itertools.islice(G_iter(data_two),0,None,1):
            try:
                #X = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '02_Generate_Database', f"{DATA_TYPE}_Data", NetworkName, f"{NetworkName}-Data_{approach}_{key}_{E}_{T}.pkl"))
                #X = UnpickleObj(os.path.join(r"D:\WORK\EN\Documents\Ecological_Restoration_04\Real_Data", f"{NetworkName}\{NetworkName}-Data_{approach}_{key}_{E}_{T}.pkl"))
                X = UnpickleObj(f"/home/udit/Documents/Ecological_Restoration_{loc}/{DATA_TYPE.split('_')[0]}_Data/{NetworkName}/{NetworkName}-Data_{approach}_{key}_{E}_{T}.pkl")
                M_per = RemovalFromNodeList(data_two[key][E],M_df,axis=key[2])
                M_per = M_per.loc[(M_per!=0).any(axis=1)] #Cleaning Row isoloates
                M_per = M_per.loc[:, (M_per!=0).any(axis=0)] #Cleaning Column isoloates
                
                M_per_dict = {}
                
                Degree_P = list(np.sum(np.array(M_per), axis = 1))
                Degree_A = list(np.sum(np.array(M_per), axis = 0))
                g_p = np.sum(P['g0'] * np.power(np.array(Degree_P),1-P['t'])) / len(Degree_P)
                g_a = np.sum(P['g0'] * np.power(np.array(Degree_A),1-P['t'])) / len(Degree_A)
                M_per_dict['g_p'] = g_p
                M_per_dict['g_a'] = g_a
                beta_eff_p, _, _ = ODE_Solve_Low(t0=0,tf=100,num=50,x_low=0,A=np.array(getProjectedNet(M_per, flag = [True, False])[0]))
                beta_eff_a, _, _ = ODE_Solve_Low(t0=0,tf=100,num=50,x_low=0,A=np.array(getProjectedNet(M_per, flag = [False, True])[1]))
                M_per_dict['beta_eff_p'] = beta_eff_p
                M_per_dict['beta_eff_a'] = beta_eff_a
                
                M_per_dict['n'] = M_per.shape[0]
                M_per_dict['m'] = M_per.shape[1]
                M_per_dict['L'] = sum(np.array(M_per).flatten())
                M_per_dict['S'] = M_per_dict['n'] + M_per_dict['m']
                M_per_dict['A'] = M_per_dict['m'] / M_per_dict['n']
                M_per_dict['C'] = M_per_dict['L'] / (M_per_dict['m']*M_per_dict['n'])
                M_per_dict['N'] = NestednessCalculator(np.array(M_per)).nodf(np.array(M_per))
                
                Z1 = np.mean(np.array(X[3][0][-1,:]))
                Z2 = np.mean(np.array(list(map(isExtinct,X[3][0][-1,:]))))
                Z3 = getResilience(X[3][0])
                Z4 = getPersistence(np.array(list(map(isExtinct,X[3][0][-1,:])))) / M_df.shape[T]
                
                temp = [M_per_dict[x] for x in ['n','m','L','S','A','C','N','g_p','g_a','beta_eff_p','beta_eff_a']] + [Z1,Z2,Z3,Z4]
                Data_Scatter += [temp]
                
                for Step in list(range(1,len(X[0])+1)):
                    M_per = M_per.append(M_df.loc[X[0][Step-1]])
                    M_per[M_per.isnull()] = M_df
                    M_per = M_per.loc[(M_per!=0).any(axis=1)] #Cleaning Row isoloates
                    M_per = M_per.loc[:, (M_per!=0).any(axis=0)] #Cleaning Column isoloates
                    
                    Degree_P = list(np.sum(np.array(M_per), axis = 1))
                    Degree_A = list(np.sum(np.array(M_per), axis = 0))
                    g_p = np.sum(P['g0'] * np.power(np.array(Degree_P),1-P['t'])) / len(Degree_P)
                    g_a = np.sum(P['g0'] * np.power(np.array(Degree_A),1-P['t'])) / len(Degree_A)
                    M_per_dict['g_p'] = g_p
                    M_per_dict['g_a'] = g_a
                    beta_eff_p, _, _ = ODE_Solve_Low(t0=0,tf=100,num=50,x_low=0,A=np.array(getProjectedNet(M_per, flag = [True, False])[0]))
                    beta_eff_a, _, _ = ODE_Solve_Low(t0=0,tf=100,num=50,x_low=0,A=np.array(getProjectedNet(M_per, flag = [False, True])[1]))
                    M_per_dict['beta_eff_p'] = beta_eff_p
                    M_per_dict['beta_eff_a'] = beta_eff_a

                    M_per_dict['n'] = M_per.shape[0]
                    M_per_dict['m'] = M_per.shape[1]
                    M_per_dict['L'] = sum(np.array(M_per).flatten())
                    M_per_dict['S'] = M_per_dict['n'] + M_per_dict['m']
                    M_per_dict['A'] = M_per_dict['m'] / M_per_dict['n']
                    M_per_dict['C'] = M_per_dict['L'] / (M_per_dict['m']*M_per_dict['n'])
                    M_per_dict['N'] = NestednessCalculator(np.array(M_per)).nodf(np.array(M_per))
                    
                    Z1 = np.mean(np.array(X[3][Step][-1,:]))
                    Z2 = np.mean(np.array(list(map(isExtinct,X[3][Step][-1,:]))))
                    Z3 = getResilience(X[3][Step])
                    Z4 = getPersistence(np.array(list(map(isExtinct,X[3][Step][-1,:])))) / M_df.shape[T]
                    
                    temp = [M_per_dict[x] for x in ['n','m','L','S','A','C','N','g_p','g_a','beta_eff_p','beta_eff_a']] + [Z1,Z2,Z3,Z4]
                    Data_Scatter += [temp]
            except:
                #print(key, E, T)
                pass
    DF = pd.DataFrame(Data_Scatter, columns = ['n','m','L','S','A','C','N','g_p','g_a','beta_eff_p','beta_eff_a','Abundance','Ob Abundance','Settling Time','Persistence'])
    #DF.drop_duplicates(inplace=True, ignore_index=True)
    FileName = os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '05_Analysis', 'Dyna_of_SR', f"{loc}", f"{NetworkName}.pkl")
    os.makedirs(os.path.dirname(FileName), exist_ok=True)
    if DF.shape[0] != 0:
        PickleObj(DF,FileName)

In [14]:
Log_Dyna = Write_Log(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '05_Analysis', 'Dyna_of_SR', 'Log_Dyna.txt'), True)
#for CASE_TO_SOLVE, loc in zip(['Real_1D', 'Syn_01', 'Syn_2D', 'Real_2D'],['01','02','03','04']):
for CASE_TO_SOLVE, loc in zip(['Real_1D'],['01']):
    DATA_TYPE = CASE_TO_SOLVE
    CASE_TO_SOLVE_init(DATA_TYPE)
    NetworkName_Solved_df = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"Network_Solved_{DATA_TYPE.split('_')[0]}.pkl"))
    M = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"M_{DATA_TYPE.split('_')[0]}.pkl"))

    if DATA_TYPE.split('_')[0] == 'Syn':
        M = {k:M[k] for k in M.keys() if int(k.split('_')[2]) == 100}
    '''
    with Parallel(n_jobs=5, verbose = 10) as parallel:
            data = parallel(delayed(get_Data)(NetworkName,loc) for NetworkName in itertools.islice(M.keys(),0,None,1))
    '''
    for NetworkName in itertools.islice(M.keys(),1,None,1):
        get_Data(NetworkName, loc)
        temp = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '05_Analysis', 'Dyna_of_SR', f"{loc}", f"{NetworkName}.pkl"))
        print(NetworkName, ': ', temp.shape)
        Log_Dyna.Log_Entry(f"{NetworkName} : {temp.shape}")
Log_Dyna.close_Log()

ipykernel_launcher:32: RuntimeWarning: invalid value encountered in double_scalars
ipykernel_launcher:33: RuntimeWarning: invalid value encountered in double_scalars
/home/udit/anaconda3/envs/master/lib/python3.7/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
ipykernel_launcher:32: RuntimeWarning: invalid value encountered in double_scalars
ipykernel_launcher:33: RuntimeWarning: invalid value encountered in double_scalars
/home/udit/anaconda3/envs/master/lib/python3.7/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
ipykernel_launcher:32: RuntimeWarning: invalid value encountered in double_scalars
ipykernel_launcher:33: RuntimeWarning: invalid value encountered in double_scalars


In [8]:
temp = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '05_Analysis', 'Dyna_of_SR', f"{loc}", f"{NetworkName}.pkl"))
print(NetworkName, ': ', temp.shape)
temp.head()

M_PL_003 :  (14830, 15)


,n,m,L,S,A,C,N,g_p,g_a,beta_eff_p,beta_eff_a,Abundance,Ob Abundance,Settling Time,Persistence
0,26,23,61.0,49,0.884615,0.102007,0.169146,1.440472,1.509306,0.588246,0.605079,0.131042,0.131042,52,0.722222
1,27,23,62.0,50,0.851852,0.099839,0.168488,1.424158,1.516766,0.573875,0.605079,0.130789,0.130789,52,0.750000
2,28,23,63.0,51,0.821429,0.097826,0.167618,1.409009,1.523821,0.560083,0.605079,0.130554,0.130554,51,0.777778
3,29,23,64.0,52,0.793103,0.095952,0.166566,1.394906,1.530532,0.546866,0.605079,0.130334,0.130334,51,0.805556
4,30,23,65.0,53,0.766667,0.094203,0.165359,1.381742,1.536944,0.534206,0.605079,0.130129,0.130129,51,0.833333


In [ ]:
for NetworkName in itertools.islice(M.keys(),0,None,1):
    temp = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '05_Analysis', 'Dyna_of_SR', f"{loc}", f"{NetworkName}.pkl"))
    print(NetworkName, ': ', temp.shape)

In [ ]:
X = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '05_Analysis', 'Dyna_of_SR', "01", "M_PL_003.pkl"))
print(X.shape)
X.head()

In [ ]:
UPDATE_SAVED = True
for CASE_TO_SOLVE, loc in zip(['Real_1D', 'Syn_1D', 'Syn_2D', 'Real_2D'],['01','02','03','04']):
    DATA_TYPE = CASE_TO_SOLVE
    CASE_TO_SOLVE_init(DATA_TYPE)
    NetworkName_Solved_df = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"Network_Solved_{DATA_TYPE.split('_')[0]}.pkl"))
    M = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"M_{DATA_TYPE.split('_')[0]}.pkl"))
    if DATA_TYPE.split('_')[0] == 'Syn':
        M = {k:M[k] for k in M.keys() if int(k.split('_')[2]) == 100}
    D = {}
    DF = pd.DataFrame([],columns = ['n','m','L','S','A','C','N','g_p','g_a','beta_eff_p','beta_eff_a','Abundance','Ob Abundance','Settling Time','Persistence'])

    for NetworkName in itertools.islice(M.keys(),0,None,1):
        X = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '05_Analysis', "Dyna_of_SR", f"{loc}", f"{NetworkName}.pkl"))
        #X = UnpickleObj(f"Dyna_of_SR/{loc}/{NetworkName}.pkl")
        X = X.apply(pd.to_numeric)
        X.drop(X[ X['Ob Abundance'] == 0 ].index ,inplace = True)
        Y1 = X.corr(method = 'pearson')
        Y2 = X.corr(method = 'kendall')
        Y3 = X.corr(method = 'spearman')
        D[NetworkName] = [Y1,Y2,Y3]
        DF = pd.concat([DF,X], ignore_index = True)
        print(DF.shape)
    del NetworkName, X, Y1, Y2, Y3, M, NetworkName_Solved_df

    DF = DF.apply(pd.to_numeric)
    DF.sort_values(by = ['Abundance', 'Ob Abundance'], axis = 0, inplace = True, ignore_index = True)
    DF.drop(DF[ DF['Ob Abundance'] == 0 ].index ,inplace = True)
    DF.reset_index(drop = True, inplace = True)
    print(DATA_TYPE, DF.shape)
    Z = DF.describe()
    Y1 = DF.corr(method = 'pearson')
    Y2 = DF.corr(method = 'kendall')
    Y3 = DF.corr(method = 'spearman')
    Data = [Z, Y1, Y2, Y3, DF, D]
    if UPDATE_SAVED:
        PickleObj(Data, os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '05_Analysis', 'Dyna_of_SR', f"Data_{loc}_Consolidated.pkl"))
del loc, CASE_TO_SOLVE, DF, Z, Y1, Y2, Y3, Data
del D, UPDATE_SAVED

In [ ]:
for CASE_TO_SOLVE, loc in zip(['Real_1D'],['01']):
    DATA_TYPE = CASE_TO_SOLVE
    CASE_TO_SOLVE_init(DATA_TYPE)
    NetworkName_Solved_df = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"Network_Solved_{DATA_TYPE.split('_')[0]}.pkl"))
    M = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"M_{DATA_TYPE.split('_')[0]}.pkl"))
    if DATA_TYPE.split('_')[0] == 'Syn':
        M = {k:M[k] for k in M.keys() if int(k.split('_')[2]) == 100}
    D = {}
    DF = pd.DataFrame([],columns = ['n','m','L','S','A','C','N','g_p','g_a','beta_eff_p','beta_eff_a','Abundance','Ob Abundance','Settling Time','Persistence'])

    for NetworkName in itertools.islice(M.keys(),0,1,1):
        X = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '05_Analysis', "Dyna_of_SR", f"{loc}", f"{NetworkName}.pkl"))
        print(X)

In [ ]:
FileName = os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '05_Analysis', 'Dyna_of_SR', "Data_01_Consolidated.pkl")
D01 = UnpickleObj(FileName)[4]
D01.drop(D01[ D01['N'] == 0 ].index ,inplace = True)
D01.dropna(axis = 0, inplace = True)
D01 = D01[D01['Abundance'] > 1.0]
D01.reset_index(drop = True, inplace = True)

FileName = os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '05_Analysis', 'Dyna_of_SR', "Data_02_Consolidated.pkl")
D02 = UnpickleObj(FileName)[4]
D02.drop(D02[ D02['N'] == 0 ].index ,inplace = True)
D02.dropna(axis = 0, inplace = True)
D02 = D02[D02['Abundance'] > 1.0]
D02.reset_index(drop = True, inplace = True)

FileName = os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '05_Analysis', 'Dyna_of_SR', "Data_03_Consolidated.pkl")
D03 = UnpickleObj(FileName)[4]
D03.drop(D03[ D03['N'] == 0 ].index ,inplace = True)
D03.dropna(axis = 0, inplace = True)
D03 = D03[D03['Abundance'] > 0.3]
D03.reset_index(drop = True, inplace = True)

FileName = os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '05_Analysis', 'Dyna_of_SR', "Data_04_Consolidated.pkl")
D04 = UnpickleObj(FileName)[4]
D04.drop(D04[ D04['N'] == 0 ].index ,inplace = True)
D04.dropna(axis = 0, inplace = True)
D04 = D04[D04['Abundance'] > 0.3]
D04.reset_index(drop = True, inplace = True)

In [ ]:
print(D01.shape)
print(D02.shape)
print(D03.shape)
print(D04.shape)

# Debugging the Code

In [ ]:
for CASE_TO_SOLVE, loc in zip(['Real_1D'],['01']):
    DATA_TYPE = CASE_TO_SOLVE
    CASE_TO_SOLVE_init(DATA_TYPE)
    NetworkName_Solved_df = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"Network_Solved_{DATA_TYPE.split('_')[0]}.pkl"))
    M = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"M_{DATA_TYPE.split('_')[0]}.pkl"))
    if DATA_TYPE.split('_')[0] == 'Syn':
        M = {k:M[k] for k in M.keys() if int(k.split('_')[2]) == 100}
Data_Scatter = []
#print(NetworkName)
NetworkName = 'M_PL_041'
M_df = M[NetworkName]
#data_two = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '02_Generate_Database', f"{DATA_TYPE}_Data", f"{NetworkName}", f"{NetworkName}-Data_two.pkl"))
#data_two = UnpickleObj(os.path.join(r"D:\WORK\EN\Documents\Ecological_Restoration_01\Real_Data", f"{NetworkName}\{NetworkName}-Data_two.pkl"))
data_two = UnpickleObj(f"/home/udit/Documents/Ecological_Restoration_{loc}/{DATA_TYPE.split('_')[0]}_Data/{NetworkName}/{NetworkName}-Data_two.pkl")
Keys_Filter = [[0,1,2], [2,5,8], [0,2]]
Ensembles_batch = [0,10]
data_two = {k: v[Ensembles_batch[0]:min(Ensembles_batch[1],len(v))] for k,v in data_two.items() if k in set(list(itertools.product(*Keys_Filter)))}
del Keys_Filter, Ensembles_batch
P = {i:j for i,j in zip(['a', 'b', 'mu', 'h', 'g0', 't', 'k'],[0.3, 1.0, 1e-4, 0.7, 1.0, 0.5, 0.0])}

def G_iter(data_two):
    for key in list(data_two.keys()):
        for ensembles_ID in range(len(data_two[key])):
            yield (key, ensembles_ID, 0)

In [ ]:
for approach in ['four']:
    for key, E, T in itertools.islice(G_iter(data_two),0,1,1):
        print(approach, key, E, T)

In [ ]:
X = UnpickleObj(f"/home/udit/Documents/Ecological_Restoration_{loc}/{DATA_TYPE.split('_')[0]}_Data/{NetworkName}/{NetworkName}-Data_{approach}_{key}_{E}_{T}.pkl")
print(type(X))

In [ ]:
M_per = RemovalFromNodeList(data_two[key][E],M_df,axis=key[2])
M_per = M_per.loc[(M_per!=0).any(axis=1)] #Cleaning Row isoloates
M_per = M_per.loc[:, (M_per!=0).any(axis=0)] #Cleaning Column isoloates
print(M_per.shape)
M_per

In [ ]:
M_per_dict = {}
            
Degree_P = list(np.sum(np.array(M_per), axis = 1))
Degree_A = list(np.sum(np.array(M_per), axis = 0))
g_p = np.sum(P['g0'] * np.power(np.array(Degree_P),1-P['t'])) / len(Degree_P)
g_a = np.sum(P['g0'] * np.power(np.array(Degree_A),1-P['t'])) / len(Degree_A)
M_per_dict['g_p'] = g_p
M_per_dict['g_a'] = g_a
beta_eff_p, _, _ = ODE_Solve_Low(t0=0,tf=100,num=50,x_low=0,A=np.array(getProjectedNet(M_per, flag = [True, False])[0]))
beta_eff_a, _, _ = ODE_Solve_Low(t0=0,tf=100,num=50,x_low=0,A=np.array(getProjectedNet(M_per, flag = [False, True])[1]))
M_per_dict['beta_eff_p'] = beta_eff_p
M_per_dict['beta_eff_a'] = beta_eff_a

M_per_dict['n'] = M_per.shape[0]
M_per_dict['m'] = M_per.shape[1]
M_per_dict['L'] = sum(np.array(M_per).flatten())
M_per_dict['S'] = M_per_dict['n'] + M_per_dict['m']
M_per_dict['A'] = M_per_dict['m'] / M_per_dict['n']
M_per_dict['C'] = M_per_dict['L'] / (M_per_dict['m']*M_per_dict['n'])
M_per_dict['N'] = NestednessCalculator(np.array(M_per)).nodf(np.array(M_per))

Z1 = np.mean(np.array(X[3][0][-1,:]))
Z2 = np.mean(np.array(list(map(isExtinct,X[3][0][-1,:]))))
Z3 = getResilience(X[3][0])
Z4 = getPersistence(np.array(list(map(isExtinct,X[3][0][-1,:])))) / M_df.shape[T]

temp = [M_per_dict[x] for x in ['n','m','L','S','A','C','N','g_p','g_a','beta_eff_p','beta_eff_a']] + [Z1,Z2,Z3,Z4]
Data_Scatter += [temp]
print(Data_Scatter)

In [ ]:
counter = 0
for Step in list(range(1,len(X[0])+1)):
    M_per = M_per.append(M_df.loc[X[0][Step-1]])
    M_per[M_per.isnull()] = M_df
    M_per = M_per.loc[(M_per!=0).any(axis=1)] #Cleaning Row isoloates
    M_per = M_per.loc[:, (M_per!=0).any(axis=0)] #Cleaning Column isoloates
    
    Degree_P = list(np.sum(np.array(M_per), axis = 1))
    Degree_A = list(np.sum(np.array(M_per), axis = 0))
    g_p = np.sum(P['g0'] * np.power(np.array(Degree_P),1-P['t'])) / len(Degree_P)
    g_a = np.sum(P['g0'] * np.power(np.array(Degree_A),1-P['t'])) / len(Degree_A)
    M_per_dict['g_p'] = g_p
    M_per_dict['g_a'] = g_a
    beta_eff_p, _, _ = ODE_Solve_Low(t0=0,tf=100,num=50,x_low=0,A=np.array(getProjectedNet(M_per, flag = [True, False])[0]))
    beta_eff_a, _, _ = ODE_Solve_Low(t0=0,tf=100,num=50,x_low=0,A=np.array(getProjectedNet(M_per, flag = [False, True])[1]))
    M_per_dict['beta_eff_p'] = beta_eff_p
    M_per_dict['beta_eff_a'] = beta_eff_a

    M_per_dict['n'] = M_per.shape[0]
    M_per_dict['m'] = M_per.shape[1]
    M_per_dict['L'] = sum(np.array(M_per).flatten())
    M_per_dict['S'] = M_per_dict['n'] + M_per_dict['m']
    M_per_dict['A'] = M_per_dict['m'] / M_per_dict['n']
    M_per_dict['C'] = M_per_dict['L'] / (M_per_dict['m']*M_per_dict['n'])
    M_per_dict['N'] = NestednessCalculator(np.array(M_per)).nodf(np.array(M_per))
    
    Z1 = np.mean(np.array(X[3][Step][-1,:]))
    Z2 = np.mean(np.array(list(map(isExtinct,X[3][Step][-1,:]))))
    Z3 = getResilience(X[3][Step])
    Z4 = getPersistence(np.array(list(map(isExtinct,X[3][Step][-1,:])))) / M_df.shape[T]
    
    temp = [M_per_dict[x] for x in ['n','m','L','S','A','C','N','g_p','g_a','beta_eff_p','beta_eff_a']] + [Z1,Z2,Z3,Z4]
    Data_Scatter += [temp]
    counter += 1
    print(counter)

In [ ]:
temp = np.array(Data_Scatter)
print(temp.shape)

In [ ]:
DF = pd.DataFrame(Data_Scatter, columns = ['n','m','L','S','A','C','N','g_p','g_a','beta_eff_p','beta_eff_a','Abundance','Ob Abundance','Settling Time','Persistence'])
print(DF.shape)
DF.head()

In [ ]:
DF.drop_duplicates(inplace=True, ignore_index=True)
print(DF.shape)
DF.head()

In [ ]:
FileName = os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '05_Analysis', 'Dyna_of_SR', f"{loc}", f"{NetworkName}.pkl")
os.makedirs(os.path.dirname(FileName), exist_ok=True)
PickleObj(DF,FileName)

In [ ]:
DF = UnpickleObj(FileName)
print(DF.shape)
DF.head()